<h1>Projeto De Previsao de Compras</h1>
<h2>Versao.: 1.2</h2>
<h2>Data.:   14/12/2018  17:20</h2>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from math import sqrt
from sklearn.metrics import mean_squared_error 

dateparse = lambda dates: pd.datetime.strptime(dates, '%d.%m.%Y %H:%M:%S')
dataFrame = pd.read_csv(r"C:\Users\Paulo Cesar\Estudos\Data Science\Ajuste Media de Compra\datasets\VENDA-GAM-343.CSV", 
                        sep=";", parse_dates = ['dt_venda'], index_col = 'dt_venda', date_parser = dateparse)

In [ ]:
dataFrame.head()

In [ ]:
def buscarDataAtual():
    data_atual = datetime.today()
    data_em_texto = '{}-{}-{}'.format(data_atual.year, data_atual.month, '01')
    return data_em_texto

In [ ]:
def calcularErro(dataAtual):
    for i, df in enumerate(arrayDeDataFrame):
        if (len(df) > 7):
            aVendida = df.iloc[-6:, 3].values
            j = 1
            while j <= 12:
                aMM = df.iloc[-8:-2, j+2].values
                erroMM = sqrt(mean_squared_error(aVendida, aMM))
                df.loc[dataAtual, 'MM'+str(j)] = erroMM
                j+=1

In [ ]:
def adicionarIndicesFaltantes(df):
    idx = pd.date_range(min(df.index), dataAtual, freq='MS')
    df.index = pd.DatetimeIndex(df.index)
    df = df.reindex(idx, fill_value=0)
    return df

In [ ]:
def calcularMediaMovel(arrayDeDataFrame):
    j = 1
    while j <= 12:
        arrayDeDataFrame[len(arrayDeDataFrame)-1]['MM'+str(j)] = arrayDeDataFrame[len(arrayDeDataFrame)-1]['qt_vendida'].rolling(j).mean()
        j += 1
    return arrayDeDataFrame

In [ ]:
#dataFrame.drop(index=dataFrame.loc[dataFrame['dt_venda'] == '2018-01-11'].index)
#df = df.drop(df[df.score < 50].index)
dataAtual = buscarDataAtual()
dataFrame = dataFrame[dataFrame.index != dataAtual]

In [ ]:
arrayMercadoria = dataFrame['cd_mercadoria'].unique()
arrayEmpresa = [1, 4, 5]
#arrayMercadoria = [117661]
#arrayEmpresa = [1]

arrayDeDataFrame = []

for emp in arrayEmpresa:
    i = 0
    for i, mer in enumerate(arrayMercadoria):
        df = dataFrame[(dataFrame['cd_mercadoria'] == mer) & 
                      (dataFrame['cd_empresa'] == emp)].sort_values(by=['dt_venda'],ascending=True)
        
        if (len(df) > 0):
            df = adicionarIndicesFaltantes(df) 
            arrayDeDataFrame.append(df)                
            arrayDeDataFrame = calcularMediaMovel(arrayDeDataFrame)
            
            
        arrayDeDataFrame[len(arrayDeDataFrame)-1].fillna(0, inplace=True)
         
calcularErro(dataAtual)

In [ ]:
arrayDeDataFrame[1]

In [ ]:
min(dict(arrayDeDataFrame[1].iloc[-1, 4:]), key=dict(arrayDeDataFrame[1].iloc[-1, 4:]).get)

In [ ]:
arrayDeDataFrame[1][min(dict(arrayDeDataFrame[1].iloc[-1, 4:]), 
                        key=dict(arrayDeDataFrame[1].iloc[-1, 4:]).get)][len(arrayDeDataFrame[1])-2]


In [ ]:
resultados = []

for i in range(0, len(arrayDeDataFrame)):
    modelo = min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)
    qt_previsao = arrayDeDataFrame[i][min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)][len(arrayDeDataFrame[i])-2]
    erro = arrayDeDataFrame[i][min(dict(arrayDeDataFrame[i].iloc[-1, 4:]), key=dict(arrayDeDataFrame[i].iloc[-1, 4:]).get)][len(arrayDeDataFrame[i])-1]
    dt_previsao = arrayDeDataFrame[i].index[-1]
    cod_mercadoria = arrayDeDataFrame[i].iloc[:,1].unique().max()
    cod_empresa = arrayDeDataFrame[i].iloc[:,0].unique().max()
    lista = [modelo, qt_previsao, dt_previsao, cod_mercadoria, cod_empresa, erro ]
    resultados.append(lista)

    
dfR = pd.DataFrame(resultados,columns=['modelo', 'qt_previsao', 'dt_previsao', 'cd_mercadoria','cd_empresa', 'RMSE'])
dfR